# Solving a Cell Tracking Challenge Dataset

This notebook walks you through solving a Cell Tracking Challenge dataset with `tracktour`, and then visualizing the results as a `Tracks` layer in napari.

Pre-requisites:

- A working installation of Gurobi

We assume you are familiar with:

- Python and Juypter notebooks
- The Cell Tracking Challenge data format
- Creating a virtual environment and installing packages (e.g. using `pip`)
- Cloning/downloading files from GitHub

## Step 1: Environment & Download

1. Download this notebook
2. Download the `sample_data` folder and place it in the same folder as this notebook. Alternatively, you can download any Cell Tracking Challenge dataset from [their website](https://celltrackingchallenge.net/2d-datasets/).
3. Create a new environment with `python >= 3.8` and activate it
4. Install tracktour and napari e.g. `pip install tracktour==0.0.5 napari[pyqt5]==0.4.19`^

^For more detailed `napari` install instructions, check out the [install guide](https://napari.org/stable/tutorials/fundamentals/installation).

## Step 2: Define Filepaths

To run tracktour on the CTC data and visualize the results, we'll need to first define the paths to the images and segmentation, and the output path to save results.

If you've placed your `sample_data` folder int he same folder as this notebook, the code below will define the correct paths for you. Alternatively, edit these paths if you've
placed your `sample_data` folder elsewhere, or downloaded a different dataset from the CTC website.

In [ ]:
import os

root = os.path.join(os.getcwd(), 'sample_data', 'PhC-C2DL-PSC')
seg_path = os.path.join(root, '02_ST', 'SEG')
im_path = os.path.join(root, '02')
out_path = os.path.join(root, '02_RES')

## Step 3: Run `tracktour`

The easiest way to run `tracktour` on Cell Tracking Challenge data is via its `ctc` utility (which can also be accessed from the command line, see the tracktour package README).

The `ctc` function takes as input the path to the segmentation, the path to save results, and an optional parameter for the number of neighbours to use. Once executed, it will save the tracking results into the folder at `out_path` in typical [Cell Tracking Challenge format](https://celltrackingchallenge.net/datasets/).

**Note**: Tracktour keeps a list of the correct scales of all Cell Tracking Challenge datasets with ST segmentations available. Tracktour will warn you if it can't find the scale for your dataset. The results of 2D datasets will not be affected by scale, but it's quite important for 3D datasets. You can read the scale of each CTC dataset by looking at `More Details` under the dataset listing on the website. To set the `tracktour` scale yourself, you must interact with the `Tracker` object. See the [toy example](./toy.py) for how to do this.

In [ ]:
from tracktour.cli import ctc
ctc(
    seg_path,
    out_path,
    k_neighbours=10, # optional
)

## Step 4: Visualize

We will load the images, segmentation and computed tracks for visualization with `napari`. If you are not familiar with `napari`, check out their [getting started guide](https://napari.org/stable/tutorials/fundamentals/getting_started.html).

The tracks will be visualized using the `Labels` [layer](https://napari.org/stable/howtos/layers/labels.html) and the `Tracks` [layer](https://napari.org/stable/howtos/layers/tracks.html) in `napari`. 

The function for converting the segmentation and tracks into a `Tracks` layer is defined below.

In [ ]:
import pandas as pd

from collections import defaultdict
from napari.layers import Tracks
from tracktour import extract_im_centers

def get_tracks(seg: 'np.ndarray', res_path: str):
    """Get tracks from a segmentation and a tracking result file."""
    track_ids = pd.read_csv(res_path, sep=' ', header=None)
    track_ids.columns = ['track_id', 'start', 'end', 'parent']
    coords, _, _, _ = extract_im_centers(seg)
    track_graph = defaultdict(list)
    for _, row in track_ids[track_ids.parent != 0].iterrows():
        track_graph[row['track_id']].append(row['parent'])
    # 3D seg means we have 2D frames, otherwise 3D
    coord_keys = ['y', 'x'] if len(seg.shape) == 3 else ['z', 'y', 'x']
    tracks_layer = Tracks(
        data=coords[['label', 't'] + coord_keys],
        graph=track_graph,
    )
    tracks_layer.name = 'tracks'
    return tracks_layer

Now that our function is defined, we can load our images, segmentation and tracks into `napari` layers.

In [ ]:
from tracktour import load_tiff_frames
images = load_tiff_frames(im_path)
result_seg = load_tiff_frames(out_path)
# convert seg to tracks layer
tracks_layer = get_tracks(result_seg, os.path.join(out_path, 'res_track.txt'))

Finally, we make a `napari` viewer and add the layers.

In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(images, name='PhC-C2DL-PSC')
viewer.add_labels(result_seg, name='Segmentation')
viewer.add_layer(tracks_layer)

Here is a GIF of the tracks computed on the sample data.

![](./ims/sample_tracks.gif)